### INFO284 Machine Learning Exam, spring 2024

#### Importing and versioncontrol for relevant libraries

In [63]:
import sys
print("Python version: {}".format(sys.version))
import pandas as pd
print("pandas version: {}".format(pd.__version__))
import matplotlib
import matplotlib.pyplot as plt
print("matplotlib version: {}".format(matplotlib.__version__))
import numpy as np
print("NumPy version: {}".format(np.__version__))
import scipy as sp
print("SciPy version: {}".format(sp.__version__))
import IPython
print("IPython version: {}".format(IPython.__version__))
import sklearn
print("scikit-learn version: {}".format(sklearn.__version__))
import seaborn as sns
print("seaborn version: {}".format(sns.__version__))

Python version: 3.12.2 (tags/v3.12.2:6abddd9, Feb  6 2024, 21:26:36) [MSC v.1937 64 bit (AMD64)]
pandas version: 2.2.1
matplotlib version: 3.8.3
NumPy version: 1.26.4
SciPy version: 1.12.0
IPython version: 8.21.0
scikit-learn version: 1.4.1.post1
seaborn version: 0.13.2


#### Importing the dataset

In [64]:
filePath = 'elektronisk-rapportering-ers-2018-fangstmelding-dca-simple.csv'
# Keep in mind that the file is encoded in UTF-8 so it will only work if you have the correct version of pandas.
df = pd.read_csv(filePath, encoding="UTF-8" , delimiter=";")
print(f"Before pre-processing the dataset has {df.shape[1]} columns and {df.shape[0]} rows")


Before pre-processing the dataset has 45 columns and 305434 rows


#### Step 1: chosing target and pre-prossesing
After taking some time to understand the data we have chosen our target features to be the catches of Hyse, Torsk and Sei. Next we must pre-prosess the data into suitable forms for modeling, this might take different forms as we try different ways to model it. The **key skill** here will therefor be the ability to manipulate the pandas dataframe.

Ways we might process the data:
1. Remove uncessary columns.
2. Collapse columns that say the same thing into one column.
3. Remove or manipulate non-sensical or missing values.
4. Transform values to work with our model.

*Question: should we also show how we went about understanding the data?*

(Old)The dataset needs a lot of pre processing. Current objective is to clean and remove redundancy and irrelevant columns. In this step we are actively discussing what our goal will be machine learning model, while understanding the data on a deeper level

#### 1.1: First purge of redundancy
In this step we will do an intial transformation of the data which will be applicable to all our modeling(maybe?).

We might also set up functions for more complicated manipulations?

In [65]:
# Excluding irrelevant columns
df = df[['Melding ID','Art - gruppe','Meldingsdato','Starttidspunkt','Havdybde start','Havdybde stopp','Rundvekt', 'Hovedområde start', 'Hovedart FAO']]


In [66]:
# Calculating the percentage of NaNs for each column to see whether we can afford to drop them
nanPercentage = df.isna().mean() * 100
print(nanPercentage)
df = df.dropna()
print("Current amount of rows: " + str(df.shape[0]))


Melding ID           0.000000
Art - gruppe         1.631122
Meldingsdato         0.000000
Starttidspunkt       0.000000
Havdybde start       0.000000
Havdybde stopp       0.000000
Rundvekt             1.629812
Hovedområde start    1.350210
Hovedart FAO         1.629812
dtype: float64
Current amount of rows: 296664


In [67]:
# Collapsing certain columns?
#
#

In [68]:
# Many of the sea depth notations are positiv, which doesn't make sense.
# But the amount of them in relation to number of entries means it can't be discounted as an error
# In the lecture on fisheries it was mentioned that a lot fo these are inputed manually
# And that most of these non-sensical sea depths are actually correct, just lacking a minus.
# Therefore we are simply flipping all the positive sea depth into negatives.
df['Havdybde start'] = -df['Havdybde start'].abs()
df['Havdybde stopp'] = -df['Havdybde stopp'].abs()

#### 1.2: Pre-prosseing for test 1
Here we will attempt to use a KNN model to predict what the main catch of each fishing trip ended up being. First we will look at the data we have.

In [69]:
print(df.shape)
df = df[df['Art - gruppe'].isin(['Torsk', 'Sei', 'Hyse'])]
print(df.shape)
df = df.pivot_table(index=['Melding ID', 'Meldingsdato', 'Starttidspunkt', 'Havdybde start', 'Havdybde stopp','Hovedområde start', 'Hovedart FAO'], columns='Art - gruppe', values='Rundvekt', aggfunc='sum').reset_index()
df = df.fillna(0) # filling Nan values with 0
print(df.shape)
df['Dominant Art'] = df[['Hyse', 'Sei', 'Torsk']].idxmax(axis=1)
print("Current amount of rows: " + str(df.shape[0]))
print(df.shape)
# sns.scatterplot(x='Havdybde start', y='Havdybde stopp', data=df, hue='Art - gruppe')


(296664, 9)
(137339, 9)
(65107, 10)
Current amount of rows: 65107
(65107, 11)


#### 2.2: Testing KNN